In [1]:
using Pkg
Pkg.activate("C:/Users/lisah/Documents/Repos/ma-code")
include("c:/Users/lisah/Documents/Repos/ma-code/src/src.jl")

  Activating project at `C:\Users\lisah\Documents\Repos\ma-code`


Main.Src

In [2]:
using GlobalSensitivity, .Src, QuasiMonteCarlo, Plots

# Tools

In [3]:
function gsa_function(p)
    hprm = Hyperprm(p[1], p[2], p[3], p[4], 1000, 0.0)
    sol = sol_klausmeier(hprm, t_fixed=true, t_end=100.0)
    return vcat(sol[!, "w"], sol[!,"n"]) # first w then n trajctory
end

gsa_function (generic function with 1 method)

In [4]:
function compute_sobol_indices_v2(lb::Vector{Float64}, ub::Vector{Float64}; samples::Int64=10000)
    sampler = SobolSample()
    A, B = QuasiMonteCarlo.generate_design_matrices(samples, lb, ub, sampler)

    res = gsa(gsa_function, Sobol(), A, B)

    w_fo, w_to = res.S1[1:1000, :], res.ST[1:1000, :]
    n_fo, n_to = res.S1[1001:end, :], res.ST[1001:end, :]
    sobol_w = (fo=w_fo, to=w_to)
    sobol_n = (fo=n_fo, to=n_to)
    
    return (sobol_w=sobol_w, sobol_n=sobol_n)
end

compute_sobol_indices_v2 (generic function with 1 method)

In [5]:
function sobol_index_subplot(sobol::NamedTuple, comp::String; title::String="")

    #colors = [:blue, :turquoise, :orange, :red]
    colors = [
        "#9ABCE4",  # w0  → TUM blue light-dark
        "#FAD080",   # n0  → TUM orange-2
        "#3070B3",  # a   → TUM blue brand
        "#F7811E",  # m   → TUM orange
    ]
    parameters = ["w0", "n0", "a", "m"];

    # very stupid way to create time axis
    hprm = Src.Hyperprm(1,1,1,1,1000,0.0)
    sol = Src.sol_klausmeier(hprm, t_fixed=true, t_end=100.0)
    times = sol[!,"time"]

    si_plot = plot(legend=:topright)

    for k in 1:4
        col = colors[k]
        prm = parameters[k]
        plot!(times, sobol.fo[:,k], label="$prm", lw=2, color=col, linestyle=:solid)
        plot!(times, sobol.to[:,k], label="", lw=2, color=col, linestyle=:dash)
    end
    #xlabel!("Time")
    ylabel!("Sobol indices $comp")
    title!(title)
    return si_plot
end

sobol_index_subplot (generic function with 1 method)

In [6]:
function sobol_index_subplot_wn(sobol_result::NamedTuple; title::String="")

    # plot n compartment
    plot_n = sobol_index_subplot(sobol_result.sobol_n, "n", title = title)

    # plot w compartment
    plot_w = sobol_index_subplot(sobol_result.sobol_w, "w")

    # compare both
    return plot(plot_n, plot_w, layout=(2,1), size=(700,700))
end

sobol_index_subplot_wn (generic function with 1 method)

# Analysis

In [7]:
# define parameter bounds by regions
# w0, n0, a, m
lb_r1 = [0.0, 0.0, 0.0, 0.45-0.45*0.1]
ub_r1 = [2.0, 2.0, 0.8, 0.45+0.45*0.1]

lb_r2 = [0.0, 0.5, 0.8, 0.45-0.45*0.1]
ub_r2 = [0.9, 1.5, 1.0, 0.45+0.45*0.1]

lb_r3 = [0.0, 0.8, 1.0, 0.45-0.45*0.1]
ub_r3 = [0.6, 2.0, 2.0, 0.45+0.45*0.1]

lb_r4 = [1.9, 0.0, 1.0, 0.45-0.45*0.1]
ub_r4 = [3.0, 0.2, 2.0, 0.45+0.45*0.1];

## Region 1

In [ ]:
r1 = compute_sobol_indices_v2(lb_r1, ub_r1, samples=50000)

## Region 2

In [ ]:
r2 = compute_sobol_indices_v2(lb_r2, ub_r2, samples=50000)

## Region 3

In [ ]:
r3 = compute_sobol_indices_v2(lb_r3, ub_r3, samples=50000)

## Region 4

In [ ]:
r4 = compute_sobol_indices_v2(lb_r4, ub_r4, samples=50000)

┌ Warning: The `generate_design_matrices(n, d, sampler, R = NoRand(), num_mats)` method does not produces true and independent QMC matrices, see [this doc warning](https://docs.sciml.ai/QuasiMonteCarlo/stable/design_matrix/) for more context. 
│     Prefer using randomization methods such as `R = Shift()`, `R = MatousekScrambling()`, etc., see [documentation](https://docs.sciml.ai/QuasiMonteCarlo/stable/randomization/)
└ @ QuasiMonteCarlo C:\Users\lisah\.julia\packages\QuasiMonteCarlo\KvLfb\src\RandomizedQuasiMonteCarlo\iterators.jl:255


(sobol_w = (fo = [1.000476399862525 0.0 0.0 0.0; 0.9912388000485322 9.09711808573836e-5 0.009315186998126116 5.395723127956085e-8; … ; 0.003236524660064901 0.010675833190263443 0.9856378549438889 -0.013987593572562487; 0.0032431888955710337 0.010683096541655416 0.9856389905860187 -0.013987516052265976], to = [1.001428628860983 0.0 0.0 0.0; 0.9922369045073417 9.090831270038206e-5 0.009069274580691025 1.4273283639175944e-9; … ; 0.011819600317360795 0.008040350589686305 0.9941352337463921 0.016702128688294064; 0.011819471840377843 0.00804021661298371 0.9941351674704383 0.01670253541532789]), sobol_n = (fo = [0.0 0.9997166108389502 0.0 0.0; 2.705781676444375e-5 0.9995935540116182 1.2637668127062075e-6 6.011361756802787e-6; … ; -3.682428809561046e-16 4.0296402845979876e-17 2.928190769003983e-17 1.863949112788293e-17; -3.5067278729587936e-16 3.8551516775444635e-17 2.787651342402729e-17 1.7888137264006075e-17], to = [0.0 0.9999078392978561 0.0 0.0; 8.923507497715457e-5 0.9998183910817859 1.87

In [ ]:
sobol_index_subplot_wn(r4, title="Region 4")